## MLflow's Model Registry

In [5]:
from mlflow.tracking import MlflowClient


MLFLOW_TRACKING_URI = "http://127.0.0.1:5000"

### Interacting with the MLflow tracking server

The `MlflowClient` object allows us to interact with...
- an MLflow Tracking Server that creates and manages experiments and runs.
- an MLflow Registry Server that creates and manages registered models and model versions. 

To instantiate it we need to pass a tracking URI and/or a registry URI

In [11]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [12]:
client.create_experiment(name="my-cool-experiment")

'3'

Let's check the latest versions for the experiment with id `1`...

In [19]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1',
    filter_string="metrics.rmse < 6.4",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [20]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: 00654d27bc124f169f7cb06f56fe07e2, rmse: 6.3184
run id: c5f0794adc0942178b667b6be49b73f6, rmse: 6.3465
run id: dd7828f090f64274943a930cf5e18f83, rmse: 6.3788
run id: 2dc66fa1f62b4ff2886f31da913c0806, rmse: 6.3888


### Interacting with the Model Registry

In this section We will use the `MlflowClient` instance to:

1. Register a new version for the experiment `nyc-taxi-regressor`
2. Retrieve the latests versions of the model `nyc-taxi-regressor` and check that a new version `4` was created.
3. Transition the version `4` to "Staging" and adding annotations to it.

In [21]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [23]:
run_id = "9c4aab4451d141ee8251e23f313ea5f0"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-regressor")

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
2025/04/09 14:01:35 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: nyc-taxi-regressor, version 3
Created version '3' of model 'nyc-taxi-regressor'.


<ModelVersion: aliases=[], creation_timestamp=1744187495287, current_stage='None', description='', last_updated_timestamp=1744187495287, name='nyc-taxi-regressor', run_id='9c4aab4451d141ee8251e23f313ea5f0', run_link='', source='mlflow-artifacts:/1/9c4aab4451d141ee8251e23f313ea5f0/artifacts/model', status='READY', status_message=None, tags={}, user_id='', version='3'>

In [28]:
model_name = "nyc-taxi-regressor"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 1, stage: Archived
version: 2, stage: Staging
version: 3, stage: Production
version: 4, stage: None


/var/folders/_5/qt38hs3n6dl58ypt2xsyjwbc0000gn/T/ipykernel_5855/669935608.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=model_name)


In [29]:
model_version = 4
new_stage = "Staging"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

/var/folders/_5/qt38hs3n6dl58ypt2xsyjwbc0000gn/T/ipykernel_5855/1718116654.py:3: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1744187737743, current_stage='Staging', description='', last_updated_timestamp=1744187919892, name='nyc-taxi-regressor', run_id='088cfce74206467aa57a8faece971683', run_link='', source='models:/nyc-taxi-regressor/1', status='READY', status_message=None, tags={}, user_id='', version='4'>

In [30]:
from datetime import datetime

date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: aliases=[], creation_timestamp=1744187737743, current_stage='Staging', description='The model version 4 was transitioned to Staging on 2025-04-09', last_updated_timestamp=1744187978640, name='nyc-taxi-regressor', run_id='088cfce74206467aa57a8faece971683', run_link='', source='models:/nyc-taxi-regressor/1', status='READY', status_message=None, tags={}, user_id='', version='4'>

### Comparing versions and selecting the new "Production" model

In the last section, we will retrieve models registered in the model registry and compare their performance on an unseen test set. The idea is to simulate the scenario in which a deployment engineer has to interact with the model registry to decide whether to update the model version that is in production or not.

These are the steps:

1. Load the test dataset, which corresponds to the NYC Green Taxi data from the month of March 2021.
2. Download the `DictVectorizer` that was fitted using the training data and saved to MLflow as an artifact, and load it with pickle.
3. Preprocess the test set using the `DictVectorizer` so we can properly feed the regressors.
4. Make predictions on the test set using the model versions that are currently in the "Staging" and "Production" stages, and compare their performance.
5. Based on the results, update the "Production" model version accordingly.


**Note: the model registry doesn't actually deploy the model to production when you transition a model to the "Production" stage, it just assign a label to that model version. You should complement the registry with some CI/CD code that does the actual deployment.**

In [35]:
from sklearn.metrics import root_mean_squared_error
import pandas as pd


def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": root_mean_squared_error(y_test, y_pred)}

In [36]:
df = read_dataframe("https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-03.parquet")

In [37]:
client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')

'/Users/ajay_gupta2/Desktop/Work/mlops-zoomcamp/02-experiment-tracking/preprocessor'

In [38]:
import pickle

with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [39]:
X_test = preprocess(df, dv)

In [40]:
target = "duration"
y_test = df[target].values

In [45]:
%time test_model(name=model_name, stage="Production", X_test=X_test, y_test=y_test)

CPU times: user 6.67 s, sys: 1.65 s, total: 8.32 s
Wall time: 8.9 s


{'rmse': 6.888474351505633}

In [46]:
%time test_model(name=model_name, stage="Staging", X_test=X_test, y_test=y_test)

CPU times: user 98.4 ms, sys: 24.8 ms, total: 123 ms
Wall time: 152 ms


{'rmse': 6.659623830022513}

In [44]:
client.transition_model_version_stage(
    name=model_name,
    version=4,
    stage="Production",
    archive_existing_versions=True
)

/var/folders/_5/qt38hs3n6dl58ypt2xsyjwbc0000gn/T/ipykernel_5855/3610659809.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1744187737743, current_stage='Production', description='The model version 4 was transitioned to Staging on 2025-04-09', last_updated_timestamp=1744188504038, name='nyc-taxi-regressor', run_id='088cfce74206467aa57a8faece971683', run_link='', source='models:/nyc-taxi-regressor/1', status='READY', status_message=None, tags={}, user_id='', version='4'>